# Sample Text2SQL Agent Walkthrough

This notebook will walk users through setting up a generic Text2SQL Agent and run it against the BirdSQL - Mini Dev Dataset (https://github.com/bird-bench/mini_dev)

#### Ensure the latest version of boto3 is shown below

##### If not then run through setup_environment.ipynb in the 0-Notebook-environment/ folder

In [4]:
!pip freeze | grep -E "boto3|pandas"

boto3==1.36.24
pandas==1.5.3


#### Load in environment variables to notebook

In [2]:
# Retrieve import path
%store -r IMPORTS_PATH

# Retrieve account info
%store -r account_id
%store -r region

# Retrieve model lists
%store -r agent_foundation_model

In [7]:
BASE_BUCKET_NAME = 'text2sql-agent'
ATHENA_RESULTS_BUCKET_NAME = 'text2sql-athena-results'
BASE_DIR = 'dev_databases'
DATABASE_NAME = 'california_schools'

# Store the variables

%store BASE_BUCKET_NAME
%store ATHENA_RESULTS_BUCKET_NAME
%store BASE_DIR
%store DATABASE_NAME


Stored 'BASE_BUCKET_NAME' (str)
Stored 'ATHENA_RESULTS_BUCKET_NAME' (str)
Stored 'BASE_DIR' (str)
Stored 'DATABASE_NAME' (str)


### Retrieve BirdSQL - Mini Dev Dataset

In [3]:
# Download .zip file to local directory

!cd 2-Sample-text2sql-agent
!wget https://bird-bench.oss-cn-beijing.aliyuncs.com/dev.zip

/bin/sh: line 0: cd: 2-Sample-text2sql-agent: No such file or directory
--2025-02-19 22:10:25--  https://bird-bench.oss-cn-beijing.aliyuncs.com/dev.zip
Resolving bird-bench.oss-cn-beijing.aliyuncs.com (bird-bench.oss-cn-beijing.aliyuncs.com)... 8.141.181.247
connected. to bird-bench.oss-cn-beijing.aliyuncs.com (bird-bench.oss-cn-beijing.aliyuncs.com)|8.141.181.247|:443... 
HTTP request sent, awaiting response... 200 OK
Length: 346207293 (330M) [application/zip]
Saving to: ‘dev.zip’

100%[======================================>] 346,207,293 16.0MB/s   in 22s    

2025-02-19 22:10:48 (14.9 MB/s) - ‘dev.zip’ saved [346207293/346207293]



In [6]:
# TODO: Load data into Amazon S3 - logic for this needs to be coded
# Unzip dev.zip and load 1 of the .sqlite files into S3 if possible/Redshift Serverless
# Keep in mind that Text2SQL Agent must be able to access data

## Set up Necessary services to run Text2SQL agent

In order to run the Text2SQL agent, we will need to setup the Athena databases to make SQL queries against. The following script will:
1. Unzip the downloaded folder
2. Create S3 buckets
3. Convert .sqlite files into individual .parquet files for each table
4. Upload to the database s3 bucket
5. Set up appropriate Athena permissions
6. Create databases in Athena

In [5]:
%run data_prep.py

working


### Convert the .sqlite files into .parquet for Athena queries

#### Step 1) unzip the dev.zip and the databases zip file within the dev folder to get the .sqlite files

In [20]:
def create_and_unzip(first_zip, new_directory, second_zip):
    notebook_dir = Path.cwd()
    new_dir = Path(new_directory)
    new_dir.mkdir(parents=True, exist_ok=True)
    print(f"Created directory: {new_dir}")
    
    # Unzip first file into new directory
    with zipfile.ZipFile(first_zip, 'r') as zip_ref:
        zip_ref.extractall(new_directory)
    print(f"Unzipped {first_zip} to {new_directory}")
    
    # Create full path to subfolder
    subdirs = [d for d in new_dir.iterdir() if d.is_dir()]

    if subdirs:
        auto_found_dir = subdirs[0]
        print(f"Found directory: {auto_found_dir}")

        second_zip_path = auto_found_dir / second_zip
        print(second_zip_path)
        if second_zip_path.exists():
            with zipfile.ZipFile(second_zip_path, 'r') as zip_ref:
                zip_ref.extractall(notebook_dir)
            print(f"Unzipped {second_zip} in {notebook_dir}")
        else:
            print(f"Could not find {second_zip} in {auto_found_dir}")

    else:
        print(f"No directories found in {new_dir}")


create_and_unzip(
    first_zip='dev.zip',
    new_directory='unzipped_dev',
    second_zip='dev_databases.zip'
)


Created directory: unzipped_dev
Unzipped dev.zip to unzipped_dev
[PosixPath('unzipped_dev/dev_20240627')]
Found directory: unzipped_dev/dev_20240627
unzipped_dev/dev_20240627/dev_databases.zip
Unzipped dev_databases.zip in /home/ec2-user/SageMaker


#### Step 2) Go through all the subfolders, convert sqlite files into parquet and upload to s3 bucket

In [23]:
#Create and setup s3 bucket

def get_account_id():
    """Get the current AWS account ID"""
    sts_client = boto3.client('sts')
    return sts_client.get_caller_identity()['Account']

def create_unique_bucket_name(base_name):
    """Create a unique bucket name using account ID and a random suffix"""
    account_id = get_account_id()
    random_suffix = uuid.uuid4().hex[:6]  # 6 character random string
    return f"{base_name}-{account_id}-{random_suffix}"

def create_s3_bucket_with_permissions(base_bucket_name, region='us-west-2'):
    """
    Create S3 bucket with a unique name and appropriate permissions
    """
    bucket_name = create_unique_bucket_name(base_bucket_name)
    s3_client = boto3.client('s3')
    s3 = boto3.resource('s3')
    
    try:
        # Create bucket
        s3_client.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={'LocationConstraint': region}
        )
        print(f"Created bucket: {bucket_name}")

        # Create bucket policy
        bucket_policy = {
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Sid": "AllowAthenaAccess",
                    "Effect": "Allow",
                    "Principal": {
                        "Service": "athena.amazonaws.com"
                    },
                    "Action": [
                        "s3:GetBucketLocation",
                        "s3:GetObject",
                        "s3:ListBucket",
                        "s3:PutObject"
                    ],
                    "Resource": [
                        f"arn:aws:s3:::{bucket_name}",
                        f"arn:aws:s3:::{bucket_name}/*"
                    ]
                },
                {
                    "Sid": "AllowCurrentUserReadWrite",
                    "Effect": "Allow",
                    "Principal": {
                        "AWS": boto3.client('sts').get_caller_identity()['Arn']
                    },
                    "Action": [
                        "s3:GetObject",
                        "s3:PutObject",
                        "s3:DeleteObject",
                        "s3:ListBucket",
                        "s3:GetBucketLocation"
                    ],
                    "Resource": [
                        f"arn:aws:s3:::{bucket_name}",
                        f"arn:aws:s3:::{bucket_name}/*"
                    ]
                }
            ]
        }

        # Convert policy to JSON string
        bucket_policy = json.dumps(bucket_policy)

        # Set bucket policy
        s3_client.put_bucket_policy(
            Bucket=bucket_name,
            Policy=bucket_policy
        )
        print("Applied bucket policy")

        return bucket_name

    except ClientError as e:
        print(f"Error: {e}")
        return None


# Configuration
BASE_BUCKET_NAME = 'text2sql-agent'
REGION = 'us-west-2'

# Create main bucket
main_bucket = create_s3_bucket_with_permissions(BASE_BUCKET_NAME, REGION)
if main_bucket:
    print("Main bucket setup complete")

else:
    print("Failed to set up main bucket")


#Create athena results bucket
athena_results_bucket_name = 'text2sql-athena-results'

# Create results bucket
athena_results_bucket = create_s3_bucket_with_permissions(athena_results_bucket_name, REGION)
if main_bucket:
    print("Main bucket setup complete")

else:
    print("Failed to set up main bucket")




Created bucket: text2sql-agent-761018876800-8a4c4e
Applied bucket policy
Main bucket setup complete


In [24]:
main_bucket, athena_results_bucket

'text2sql-agent-761018876800-8a4c4e'

In [27]:
# Now go through all the subfolders to find .sqlite files to convert to .parquet files and upload to s3

def process_database_and_upload(database_folder, bucket_name):
    """
    Process SQLite database, convert to parquet, and upload to S3
    All tables from same database go to same folder
    """
    folder_path = Path(database_folder)
    database_name = folder_path.name
    
    # Find SQLite file
    sqlite_files = list(folder_path.glob('*.db')) + list(folder_path.glob('*.sqlite'))
    if not sqlite_files:
        print(f"No SQLite file found in {database_folder}")
        return
    
    sqlite_file = sqlite_files[0]
    print(f"\nProcessing database: {database_name}")
    print(f"SQLite file: {sqlite_file}")
    
    try:
        # Connect to SQLite database
        conn = sqlite3.connect(sqlite_file)
        
        # Get list of all tables
        cursor = conn.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        
        # Create S3 client
        s3_client = boto3.client('s3')
        
        # Process each table
        for table in tables:
            table_name = table[0]
            print(f"\nProcessing table: {table_name}")
            
            try:
                # Read table into DataFrame
                df = pd.read_sql_query(f'SELECT * FROM "{table_name}"', conn)
                
                # Create local parquet file
                parquet_filename = f"{table_name}.parquet"
                local_parquet_path = folder_path / parquet_filename
                df.to_parquet(local_parquet_path, index=False)
                
                # Upload to S3 (all tables in same database folder)
                s3_key = f"{database_name}/{parquet_filename}"
                s3_client.upload_file(
                    str(local_parquet_path),
                    bucket_name,
                    s3_key
                )
                print(f"Uploaded to s3://{bucket_name}/{s3_key}")
                
                # Clean up local parquet file
                os.remove(local_parquet_path)
                
            except Exception as e:
                print(f"Error processing table {table_name}: {str(e)}")
                continue
        
        conn.close()
        
    except Exception as e:
        print(f"Error processing database {database_name}: {str(e)}")
        return


BASE_DIR = 'dev_databases'
BUCKET_NAME = main_bucket

# Process each database folder
base_path = Path(BASE_DIR)
for database_folder in base_path.iterdir():
    if database_folder.is_dir():
        process_database_and_upload(database_folder, BUCKET_NAME)

print("Upload complete")


Processing database: toxicology
SQLite file: dev_databases/toxicology/toxicology.sqlite

Processing table: atom
Uploaded to s3://text2sql-agent-761018876800-8a4c4e/toxicology/atom.parquet

Processing table: bond
Uploaded to s3://text2sql-agent-761018876800-8a4c4e/toxicology/bond.parquet

Processing table: connected
Uploaded to s3://text2sql-agent-761018876800-8a4c4e/toxicology/connected.parquet

Processing table: molecule
Uploaded to s3://text2sql-agent-761018876800-8a4c4e/toxicology/molecule.parquet

Processing database: student_club
SQLite file: dev_databases/student_club/student_club.sqlite

Processing table: event
Uploaded to s3://text2sql-agent-761018876800-8a4c4e/student_club/event.parquet

Processing table: major
Uploaded to s3://text2sql-agent-761018876800-8a4c4e/student_club/major.parquet

Processing table: zip_code
Uploaded to s3://text2sql-agent-761018876800-8a4c4e/student_club/zip_code.parquet

Processing table: attendance
Uploaded to s3://text2sql-agent-761018876800-8a4c4

#### Step 3) Create external tables for each table in Athena

In [68]:
# update current role to include permissions for athena 
def add_athena_permissions(data_bucket, results_bucket):
    iam = boto3.client('iam')
    sts = boto3.client('sts')

    # Get current user/role info
    caller_identity = sts.get_caller_identity()
    current_user_arn = caller_identity['Arn']
    account_id = caller_identity['Account']
    region = boto3.session.Session().region_name

    # Define the policy
    athena_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": [
                    "athena:*"
                ],
                "Resource": [
                    f"arn:aws:athena:*:{account_id}:capacity-reservation/*",
                    f"arn:aws:athena:*:{account_id}:workgroup/primary",
                    f"arn:aws:athena:*:{account_id}:datacatalog/*"
                ]
            },

            {
                # Data bucket permissions (read-only)
                "Effect": "Allow",
                "Action": [
                    "s3:GetBucketLocation",
                    "s3:GetObject",
                    "s3:ListBucket"
                ],
                "Resource": [
                    f"arn:aws:s3:::{data_bucket}",
                    f"arn:aws:s3:::{data_bucket}/*"
                ]
            },
            {
                # Results bucket permissions (read and write)
                "Effect": "Allow",
                "Action": [
                    "s3:GetBucketLocation",
                    "s3:GetObject",
                    "s3:ListBucket",
                    "s3:PutObject",
                    "s3:ListMultipartUploadParts",
                    "s3:AbortMultipartUpload"
                ],
                "Resource": [
                    f"arn:aws:s3:::{results_bucket}",
                    f"arn:aws:s3:::{results_bucket}/*"
                ]
            },
            {
                "Effect": "Allow",
                "Action": [
                    "glue:CreateDatabase",
                    "glue:DeleteDatabase",
                    "glue:GetCatalog",
                    "glue:GetCatalogs",
                    "glue:GetDatabase",
                    "glue:GetDatabases",
                    "glue:UpdateDatabase",
                    "glue:CreateTable",
                    "glue:DeleteTable",
                    "glue:BatchDeleteTable",
                    "glue:UpdateTable",
                    "glue:GetTable",
                    "glue:GetTables",
                    "glue:BatchCreatePartition",
                    "glue:CreatePartition",
                    "glue:DeletePartition",
                    "glue:BatchDeletePartition",
                    "glue:UpdatePartition",
                    "glue:GetPartition",
                    "glue:GetPartitions",
                    "glue:BatchGetPartition",
                    "glue:StartColumnStatisticsTaskRun",
                    "glue:GetColumnStatisticsTaskRun",
                    "glue:GetColumnStatisticsTaskRuns",
                    "glue:GetCatalogImportStatus"
                ],
                "Resource": [
                    f"arn:aws:glue:*:{account_id}:catalog",
                    f"arn:aws:glue:*:{account_id}:database/*",
                    f"arn:aws:glue:*:{account_id}:table/*"
                ]
            }
        ]
    }

    try:
        # Delete existing policy if it exists
        try:
            existing_policies = iam.list_policies(Scope='Local')['Policies']
            for policy in existing_policies:
                if policy['PolicyName'] == 'AthenaQueryPermissions':
                    iam.delete_policy(PolicyArn=policy['Arn'])
                    print("Deleted existing policy")
        except Exception as e:
            print(f"Note: {e}")

        # Create the policy
        response = iam.create_policy(
            PolicyName='AthenaQueryPermissions',
            PolicyDocument=json.dumps(athena_policy)
        )
        policy_arn = response['Policy']['Arn']
        print(f"Created policy: {policy_arn}")

        # Attach the policy to the current user/role
        if ':user/' in current_user_arn:
            username = current_user_arn.split('/')[-1]
            try:
                iam.detach_user_policy(
                    UserName=username,
                    PolicyArn=policy_arn
                )
            except:
                pass
            iam.attach_user_policy(
                UserName=username,
                PolicyArn=policy_arn
            )
            print(f"Attached policy to user: {username}")
        elif ':assumed-role/' in current_user_arn:
            role_name = current_user_arn.split('/')[-2]
            try:
                iam.detach_role_policy(
                    RoleName=role_name,
                    PolicyArn=policy_arn
                )
            except:
                pass
            iam.attach_role_policy(
                RoleName=role_name,
                PolicyArn=policy_arn
            )
            print(f"Attached policy to role: {role_name}")

        print("Successfully added Athena permissions with specific bucket access")
        return True

    except Exception as e:
        print(f"Error adding permissions: {e}")
        return False

# Usage
data_bucket = main_bucket      # Bucket with parquet files
results_bucket = athena_results_bucket # Bucket for Athena query results

add_athena_permissions(data_bucket, results_bucket)


Deleted existing policy
Created policy: arn:aws:iam::761018876800:policy/AthenaQueryPermissions
Attached policy to role: biomakeragent-AgentBuildNest-SageMakerExecutionRole-kvD8zSoPKC2k
Successfully added Athena permissions with specific bucket access


True

In [70]:
#update the result bucket location for workgroup primary

def set_athena_result_location(result_bucket):
    """
    Set the query result location for Athena's primary workgroup
    """
    try:
        athena_client = boto3.client('athena')
        
        # Format the S3 path
        s3_output_location = f's3://{result_bucket}/athena-results/'
        
        # Update primary workgroup configuration
        response = athena_client.update_work_group(
            WorkGroup='primary',
            ConfigurationUpdates={
                'ResultConfigurationUpdates': {
                    'OutputLocation': s3_output_location
                },
                'EnforceWorkGroupConfiguration': True
            }
        )
        
        print(f"Successfully set Athena query result location to: {s3_output_location}")
        return True
        
    except Exception as e:
        print(f"Error setting result location: {e}")
        return False

# Usage
result_bucket = athena_results_bucket # Replace with your bucket name
set_athena_result_location(result_bucket)


Successfully set Athena query result location to: s3://text2sql-athena-results-761018876800-fc255e/athena-results/


True

In [72]:
# loop through all the folders in s3 bucket and create the database and tables in athena

import time

def list_s3_folders_and_files(bucket_name):
    """Get all database folders and their parquet files"""
    s3_client = boto3.client('s3')
    
    try:
        # Get all objects in bucket
        paginator = s3_client.get_paginator('list_objects_v2')
        database_tables = {}
        
        for page in paginator.paginate(Bucket=bucket_name):
            if 'Contents' not in page:
                continue
                
            for obj in page['Contents']:
                # Split the key into parts
                parts = obj['Key'].split('/')
                
                # Check if it's a parquet file
                if len(parts) >= 2 and parts[-1].endswith('.parquet'):
                    database_name = parts[0]
                    table_name = parts[-1].replace('.parquet', '')
                    
                    if database_name not in database_tables:
                        database_tables[database_name] = []
                    database_tables[database_name].append(table_name)
        
        return database_tables
    
    except ClientError as e:
        print(f"Error listing S3 contents: {e}")
        return None

def generate_and_create_table(results_bucket_name, parquet_bucket_name, database_name, table_name):
    """Generate and create a single table"""
    try:
        # Generate DDL
        s3_path = f's3://{parquet_bucket_name}/{database_name}/{table_name}.parquet'
        df = pd.read_parquet(s3_path)
        
        # Map pandas types to Athena types
        type_mapping = {
            'object': 'string',
            'int64': 'int',
            'float64': 'double',
            'bool': 'boolean',
            'datetime64[ns]': 'timestamp'
        }
        
        # Generate column definitions
        columns = []
        for col, dtype in df.dtypes.items():
            athena_type = type_mapping.get(str(dtype), 'string')
            columns.append(f"`{col}` {athena_type}")
        
        # Create DDL statement
        column_definitions = ',\n    '.join(columns)
        s3_location = f's3://{parquet_bucket_name}/{database_name}/'
        
        ddl = f"""CREATE EXTERNAL TABLE IF NOT EXISTS {database_name}.{table_name} (
        {column_definitions}
    )
    STORED AS PARQUET
    LOCATION '{s3_location}';"""
        
        print(f"\nGenerating table: {database_name}.{table_name}")
        
        # Execute DDL
        athena_client = boto3.client('athena')
        
        # Create table
        response = athena_client.start_query_execution(
            QueryString=ddl,
            QueryExecutionContext={
                'Database': database_name
            },
            ResultConfiguration={
                'OutputLocation': f's3://{results_bucket_name}/athena-results/'
            }
        )
        
        # Wait for table creation
        query_execution_id = response['QueryExecutionId']
        while True:
            response = athena_client.get_query_execution(QueryExecutionId=query_execution_id)
            state = response['QueryExecution']['Status']['State']
            if state in ['SUCCEEDED', 'FAILED', 'CANCELLED']:
                break
            time.sleep(1)
        
        if state == 'SUCCEEDED':
            print(f"Created table: {database_name}.{table_name}")
            return True
        else:
            print(f"Failed to create table {database_name}.{table_name}: {state}")
            return False
            
    except Exception as e:
        print(f"Error creating table {database_name}.{table_name}: {e}")
        return False

def create_all_databases_and_tables(results_bucket_name, parquet_bucket_name):
    """Create all databases and tables from S3 bucket structure"""
    try:
        # Get database and table structure from S3
        database_tables = list_s3_folders_and_files(parquet_bucket_name)
        if not database_tables:
            print("No databases/tables found in S3")
            return False
            
        athena_client = boto3.client('athena')
        
        # Process each database
        for database_name, tables in database_tables.items():
            print(f"\nProcessing database: {database_name}")
            
            # Create database
            create_database = f"CREATE DATABASE IF NOT EXISTS {database_name}"
            response = athena_client.start_query_execution(
                QueryString=create_database,
                ResultConfiguration={
                    'OutputLocation': f's3://{results_bucket_name}/athena-results/'
                }
            )
            
            # Wait for database creation
            query_execution_id = response['QueryExecutionId']
            while True:
                response = athena_client.get_query_execution(QueryExecutionId=query_execution_id)
                state = response['QueryExecution']['Status']['State']
                if state in ['SUCCEEDED', 'FAILED', 'CANCELLED']:
                    break
                time.sleep(1)
            
            if state == 'SUCCEEDED':
                print(f"Created database: {database_name}")
                
                # Create each table in the database
                for table_name in tables:
                    generate_and_create_table(
                        results_bucket_name,
                        parquet_bucket_name,
                        database_name,
                        table_name
                    )
            else:
                print(f"Failed to create database {database_name}: {state}")
                continue
        
        return True
        
    except Exception as e:
        print(f"Error in create_all_databases_and_tables: {e}")
        return False

results_bucket_name = athena_results_bucket
parquet_bucket_name = main_bucket

success = create_all_databases_and_tables(results_bucket_name, parquet_bucket_name)
if success:
    print("\nCompleted creating all databases and tables in Athena!")



Processing database: california_schools
Created database: california_schools

Generating table: california_schools.frpm
Created table: california_schools.frpm

Generating table: california_schools.satscores
Created table: california_schools.satscores

Generating table: california_schools.schools
Created table: california_schools.schools

Processing database: card_games
Created database: card_games

Generating table: card_games.cards
Created table: card_games.cards

Generating table: card_games.foreign_data
Created table: card_games.foreign_data

Generating table: card_games.legalities
Created table: card_games.legalities

Generating table: card_games.rulings
Created table: card_games.rulings

Generating table: card_games.set_translations
Created table: card_games.set_translations

Generating table: card_games.sets
Created table: card_games.sets

Generating table: card_games.sqlite_sequence
Created table: card_games.sqlite_sequence

Processing database: codebase_community
Created datab

### Create Text2SQL Agent

In [88]:
# get boto3 clients for required AWS services
sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
s3_client = boto3.client('s3')
lambda_client = boto3.client('lambda')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

In [89]:
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
region, account_id

('us-west-2', '761018876800')

In [105]:
# Generate random prefix for unique IAM roles, agent name and S3 Bucket and 
# assign variables
suffix = f"{region}-{account_id}"
agent_name = "sample-text2sql-agent"
agent_alias_name = "sample-alias"
bucket_name = f'{agent_name}-{suffix}'
bucket_key = f'{agent_name}-schema.json'
schema_name = 'sample_text2sql_agent_openapi_schema.json'
schema_arn = f'arn:aws:s3:::{bucket_name}/{bucket_key}'
bedrock_agent_bedrock_allow_policy_name = f"{agent_name}-allow-{suffix}"
bedrock_agent_s3_allow_policy_name = f"{agent_name}-s3-allow-{suffix}"
lambda_role_name = f'{agent_name}-lambda-role-{suffix}'
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{suffix}'
lambda_code_path = "lambda_function.py"
lambda_name = f'{agent_name}-{suffix}'
model_id = "anthropic.claude-3-5-sonnet-20241022-v2:0"


#### Create S3 bucket and upload API Schema

In [92]:
# Create S3 bucket for Open API schema
if region == "us-east-1":
    s3bucket = s3_client.create_bucket(
        Bucket=bucket_name
    )
else:
    s3bucket = s3_client.create_bucket(
        Bucket=bucket_name,
        CreateBucketConfiguration={ 'LocationConstraint': region } 
    )

In [113]:
#Upload Open API Schema to s3 bucket
s3_client.upload_file(schema_name, bucket_name, bucket_key)


#### Create Lambda function for Action Group

In [97]:
# Create IAM Role for the Lambda function
try:
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }

    assume_role_policy_document_json = json.dumps(assume_role_policy_document)

    lambda_iam_role = iam_client.create_role(
        RoleName=lambda_role_name,
        AssumeRolePolicyDocument=assume_role_policy_document_json
    )

    # Pause to make sure role is created
    time.sleep(10)
except:
    lambda_iam_role = iam_client.get_role(RoleName=lambda_role_name)

iam_client.attach_role_policy(
    RoleName=lambda_role_name,
    PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
)

{'ResponseMetadata': {'RequestId': 'f9e17745-6ef6-4225-8a6b-c211c177acb1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 14 Feb 2025 21:22:07 GMT',
   'x-amzn-requestid': 'f9e17745-6ef6-4225-8a6b-c211c177acb1',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

In [103]:
# Package up the lambda function code
s = BytesIO()
z = zipfile.ZipFile(s, 'w')
z.write(lambda_code_path)
z.close()
zip_content = s.getvalue()

# Create Lambda Function
lambda_function = lambda_client.create_function(
    FunctionName=lambda_name,
    Runtime='python3.12',
    Timeout=180,
    Role=lambda_iam_role['Role']['Arn'],
    Code={'ZipFile': zip_content},
    Handler='lambda_function.lambda_handler'
)

#### Create Agent

In [106]:
# Create IAM policies for agent

bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:{region}::foundation-model/{model_id}"
            ]
        }
    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

agent_bedrock_policy = iam_client.create_policy(
    PolicyName=bedrock_agent_bedrock_allow_policy_name,
    PolicyDocument=bedrock_policy_json
)

In [107]:
#Policy that allows fetching of agent's OpenAPI schema from S3

bedrock_agent_s3_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AllowAgentAccessOpenAPISchema",
            "Effect": "Allow",
            "Action": ["s3:GetObject"],
            "Resource": [
                schema_arn
            ]
        }
    ]
}


bedrock_agent_s3_json = json.dumps(bedrock_agent_s3_allow_policy_statement)
agent_s3_schema_policy = iam_client.create_policy(
    PolicyName=bedrock_agent_s3_allow_policy_name,
    Description=f"Policy to allow invoke Lambda that was provisioned for it.",
    PolicyDocument=bedrock_agent_s3_json
)

In [108]:
# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
agent_role = iam_client.create_role(
    RoleName=agent_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)
    
iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_bedrock_policy['Policy']['Arn']
)

iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_s3_schema_policy['Policy']['Arn']
)

{'ResponseMetadata': {'RequestId': 'da2e77fa-3d49-4299-b255-e78f2f1fee76',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 17 Feb 2025 21:26:54 GMT',
   'x-amzn-requestid': 'da2e77fa-3d49-4299-b255-e78f2f1fee76',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

In [123]:
agent_foundation_model = "anthropic.claude-3-5-sonnet-20241022-v2:0"
agent_description = "Text2SQL agent to run against Bird-SQL Mini-Dev benchmark dataset"
agent_instruction = """
You are an AI Agent specialized in generating SQL queries for a database. 
Your primary task is to interpret user queries, 
generate appropriate SQL queries, and provide relevant answer based on the data. 
Use only the appropriate tools as required by the specific question. Follow these instructions carefully: 
1. Before generating any SQL query, use the /getschema tool to familiarize yourself with the database structure. 
This will ensure your queries are correctly formatted and target the appropriate columns. 
2. When generating an SQL query: a. Write the query as a single line, removing all newline ("
") characters. 
b. Column names should remain consistent, do not modify the column names in the generated SQL query. 
3. When providing your response: a. Start with a brief summary of your understanding of 
the user's query. b. Explain the steps you're taking to address the query. c. Ask for clarifications from the 
user if required.
"""

response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    description=agent_description,
    instruction=agent_instruction,
    foundationModel=agent_foundation_model,
    agentResourceRoleArn=agent_role['Role']['Arn']
)


In [124]:
agent_id = response['agent']['agentId']
agent_arn = response['agent']['agentArn']
agent_id , agent_arn

('AQIMYSEPOK', 'arn:aws:bedrock:us-west-2:761018876800:agent/AQIMYSEPOK')

#### Create Agent Action Group

In [125]:
# Pause to make sure agent is created
# time.sleep(30)
# Now, we can configure and create an action group here:

print(lambda_function['FunctionArn'])
print(bucket_name, bucket_key)

# Use agent helper to clean up
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
        # 'roleArn': 'arn:aws:iam::761018876800:role/sample-text2sql-agent-lambda-role-us-west-2-761018876800',  # Add your IAM role ARN here
        # 'roleSessionName': 'BedrockAgentSession'  # Add a session name
    },
    actionGroupName='QueryAthena',
    apiSchema={
        's3': {
            's3BucketName': bucket_name,
            's3ObjectKey': bucket_key
        }
    },
    description='Execute SQL queries in Athena database'
)



arn:aws:lambda:us-west-2:761018876800:function:sample-text2sql-agent-us-west-2-761018876800
sample-text2sql-agent-us-west-2-761018876800 sample-text2sql-agent-schema.json


In [126]:
# Create allow invoke permission on lambda
response = lambda_client.add_permission(
    FunctionName=lambda_name,
    StatementId='allow_bedrock',
    Action='lambda:InvokeFunction',
    Principal='bedrock.amazonaws.com',
    SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
)

#### Prepare Agent

In [127]:
agent_prepare = bedrock_agent_client.prepare_agent(agentId=agent_id)
agent_prepare

{'ResponseMetadata': {'RequestId': '194cea38-3947-4aee-ad01-3b86d494664b',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Mon, 17 Feb 2025 21:53:32 GMT',
   'content-type': 'application/json',
   'content-length': '119',
   'connection': 'keep-alive',
   'x-amzn-requestid': '194cea38-3947-4aee-ad01-3b86d494664b',
   'x-amz-apigw-id': 'GJhqhGNGvHcEADA=',
   'x-amzn-trace-id': 'Root=1-67b3afdc-63233e2f37f3a60e3d0eacee'},
  'RetryAttempts': 0},
 'agentId': 'AQIMYSEPOK',
 'agentStatus': 'PREPARING',
 'agentVersion': 'DRAFT',
 'preparedAt': datetime.datetime(2025, 2, 17, 21, 53, 32, 819336, tzinfo=tzlocal())}

#### Create Agent Alias

In [128]:
# Pause to make sure agent is prepared
# time.sleep(30)
agent_alias = bedrock_agent_client.create_agent_alias(
    agentId=agent_id,
    agentAliasName=agent_alias_name
)

#### Invoke Agent

In [130]:
# time.sleep(30)
# Extract the agentAliasId from the response
agent_alias_id = agent_alias['agentAlias']['agentAliasId']

agent_alias_id

'T9J04ATBNJ'

In [ ]:
## create a random id for session initiator id
session_id:str = str(uuid.uuid1())

# invoke the agent API
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText="what are the open claims?",
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=True,
    endSession=False,
    sessionState={}
)


print("Request sent to Agent:\n{}".format(response))
print("====================")
print("Agent processing query now")
print("====================")

# Initialize an empty string to store the answer
answer = ""

# Iterate through the event stream
for event in response['completion']:
    # Check if the event is a 'chunk' event
    if 'chunk' in event:
        chunk_obj = event['chunk']
        if 'bytes' in chunk_obj:
            # Decode the bytes and append to the answer
            chunk_data = chunk_obj['bytes'].decode('utf-8')
            answer += chunk_data

# Now 'answer' contains the full response from the agent
print("Agent Answer: {}".format(answer))
print("====================")


### Create input .json file for all Text2SQL using ground truth provided by BirdSQL Mini-Dev dataset

In [8]:
print(ATHENA_RESULTS_BUCKET_NAME, DATABASE_NAME)

text2sql-athena-results california_schools


In [ ]:
import boto3
import time
import pandas as pd
import json
from typing import Dict, List, Any

def run_query(query: str, athena_client):
    """
    Run Athena query and return results as pandas DataFrame
    """
    
    try:
        response = athena_client.start_query_execution(
            QueryString=query,
            QueryExecutionContext={
                'Database': DATABASE_NAME
            },
            ResultConfiguration={
                'OutputLocation': f's3://{ATHENA_RESULTS_BUCKET_NAME}/athena-results/'
            }
        )
        
        query_execution_id = response['QueryExecutionId']
        
        while True:
            response = athena_client.get_query_execution(QueryExecutionId=query_execution_id)
            state = response['QueryExecution']['Status']['State']
            
            if state in ['SUCCEEDED', 'FAILED', 'CANCELLED']:
                break
                
            time.sleep(1)
            
        if state == 'SUCCEEDED':
            response = athena_client.get_query_results(QueryExecutionId=query_execution_id)
            # Process data
            results = []
            for row in response['ResultSet']['Rows'][1:]:  # Skip header row
                for field in row['Data']:
                    value = field.get('VarCharValue', '')
                    
                    results.append(str(value))
            
            # Join results with commas
            return ', '.join(results)
        else:
            print(f"Query failed with state: {state}")
            return None
            
    except Exception as e:
        print(f"Error running query: {e}")
        return None

def generate_ground_truth_answer(question_id: int, question: str, sql_query: str, 
                               sql_context: str, query_results: str) -> Dict:
    """
    Generate ground truth answer using AWS Bedrock based on question and query results
    """
    bedrock_runtime = boto3.client(
        service_name='bedrock-runtime'
    )
        
    # Construct prompt for Bedrock
    prompt = f"""You are generating ground truth answers that will be used to evaluate the factual correctness of Text2SQL agent responses.

Question: {question}
Query Results: {query_results}

Generate a natural language answer that:
1. States all numerical values and facts from the query results explicitly
2. Uses consistent formatting for numbers (maintain exact precision from results)
3. Includes all relevant values if multiple results are returned
4. States the answer in a clear, declarative way that directly addresses the question
5. Avoids additional interpretations or information not present in the query results

Remember:
- Focus only on the facts present in the query results
- Use the exact numbers shown in the results
- Structure the answer to make fact-checking straightforward
- Be explicit about any percentages, counts, or measurements
- Make sure every number in the query results is mentioned in your answer

Your answer should be easy to compare with other responses for factual accuracy."""

    # Create request body for Claude model
    body = json.dumps({
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 512,
        "temperature": 0.5,
        "messages": [
            {
                "role": "user",
                "content": [{"type": "text", "text": prompt}],
            }
        ],
    })

    try:
        # Call Bedrock
        response = bedrock_runtime.invoke_model(
            modelId='anthropic.claude-3-5-sonnet-20241022-v2:0',  # or your preferred model
            body=body
        )
        
        # Parse response
        response_body = json.loads(response['body'].read())
        answer = response_body['content'][0]['text']
        
        # Format the response in the required structure
        formatted_response = {
            "question_id": question_id,
            "question": question,
            "question_type": "TEXT2SQL",
            "ground_truth": {
                "ground_truth_sql_query": sql_query,
                "ground_truth_sql_context": sql_context,
                "ground_truth_query_result": query_results,
                "ground_truth_answer": answer
            }
        }
        
        return formatted_response
        
    except Exception as e:
        print(f"Error generating answer: {e}")
        return None

        
def get_schema(athena_client):
    """
    Get schema information for all tables in Athena databases
    """

    sql = f"""
        SELECT
            table_name,
            column_name,
            data_type
        FROM information_schema.columns
        WHERE table_schema = '{DATABASE_NAME}'
        ORDER BY table_name, ordinal_position;
        """
        
    try:
        # Start query execution
        response = athena_client.start_query_execution(
            QueryString=sql,
            QueryExecutionContext={
                'Database': DATABASE_NAME
            }
        )
            
        query_execution_id = response['QueryExecutionId']
            
        def wait_for_query_completion(query_execution_id):
            while True:
                response = athena_client.get_query_execution(
                    QueryExecutionId=query_execution_id
                )
                state = response['QueryExecution']['Status']['State']
                
                if state in ['SUCCEEDED', 'FAILED', 'CANCELLED']:
                    return state
                    
                time.sleep(2)
            
        # Wait for query completion
        state = wait_for_query_completion(query_execution_id)

        if state == 'SUCCEEDED':
            # Get query results
            results = athena_client.get_query_results(
                QueryExecutionId=query_execution_id
            )
            # Assuming you have a database connection and cursor setup
            # cursor.execute(sql)
            # results = cursor.fetchall()
            
            database_structure = []
            table_dict = {}

            # Skip the header row
            rows = results['ResultSet']['Rows'][1:]

            for row in rows:
                # Extract values from the Data structure
                table_name = row['Data'][0]['VarCharValue']
                column_name = row['Data'][1]['VarCharValue']
                data_type = row['Data'][2]['VarCharValue']
                
                # Initialize table if not exists
                if table_name not in table_dict:
                    table_dict[table_name] = []
                
                # Append column information
                table_dict[table_name].append((column_name, data_type))

            # Convert to the desired format
            for table_name, columns in table_dict.items():
                database_structure.append({
                    "table_name": table_name,
                    "columns": columns
                })

            return database_structure

        else:
            raise Exception(f"Query failed with state: {state}")
    except Exception as e:
            print(f"Error getting schema: {e}")
            raise

def generate_dataset(input_file: str, output_file: str, athena_client):
    """
    Generate dataset with ground truth answers in trajectory format
    """
    try:
        # Read input file
        with open(input_file, 'r') as f:
            questions_data = json.load(f)
        
        # Initialize trajectories dictionary
        trajectories = {}
        
        # Process each question
        for idx, item in enumerate(questions_data):
            question_id = item.get('question_id', 0)
            question = item['question']
            sql_query = item['SQL']
            
            print(f"\nProcessing question {question_id}: {question}")
            
            # Get table schema
            sql_context = get_schema(athena_client)
            # Run query
            query_results = run_query(sql_query.replace('`','"'),athena_client)
            if query_results is not None:
                # Generate answer with formatted response
                response = generate_ground_truth_answer(
                    question_id=question_id,
                    question=question,
                    sql_query=sql_query,
                    sql_context=str(sql_context),
                    query_results=query_results
                )
                
                if response:
                    # Create trajectory key
                    trajectory_key = f"Trajectory{idx + 1}"
                    
                    # Format the response for this trajectory
                    trajectory_response = [response]
                    
                    # Add to trajectories dictionary
                    trajectories[trajectory_key] = trajectory_response
                    print(f"Generated ground truth for question {question_id}")
            
        # Write results to output file
        with open(output_file, 'w') as f:
            json.dump(trajectories, f, indent=2)
            
        print(f"\nProcessed {len(trajectories)} questions. Results saved to {output_file}")
        
    except Exception as e:
        print(f"Error generating dataset: {e}")

# TODO: Alter
INPUT_FILE = "questions.json"
OUTPUT_FILE = "dataset_with_answers.json"
athena_client = boto3.client('athena')

generate_dataset(INPUT_FILE, OUTPUT_FILE,athena_client)
